# Guía 7 - Ejercicio 1.5
En este ejercicio se carga la imagen de un mosquito y se le agrega los ruidos impulsivo unimodal y gaussiano con media cero. Luego, se testea la robustez de los distintos métodos de detección de bordes vistos en la teoría.

A través de probar distintos valores en las trackbars se llegaron a varias conclusiones:
- En primer lugar, sería de gran utilidad usar algun método de pre-procesamiento, tal que elimine el el ruido gaussiano e impulsivo unimodal, como puede ser el método de fastNlDenoising, esto facilitaría la tarea de los operadores gradiente para detectar bordes.
- También se pudo comprobar como al agregar un poco de desviación estandar al ruido gaussiano con media cero, ningun detecto de bordes pudo hacer bien la detección.
- Se pudo observar como la suavización de la imagen que realiza canny, sumado a los parámetros de umbral, lo hacen el más efectivo contra ruido.

## Imports y funciones

In [3]:
import numpy as np
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
import cv2 as cv

def sobel_deteccion_bordes(imagen, ddepth=cv.CV_8U, dx=1, dy=0, ksize=3):
    if ksize == -1:
        sobel_image = cv.Scharr(imagen,ddepth,dx,dy)
    else:
        sobel_image = cv.Sobel(imagen, ddepth, dx, dy, ksize=ksize)
    return sobel_image

def ruido_impulsivo(imagen, nivel_ruido):
    noisy_image = np.copy(imagen)
    height, width = imagen.shape
    num_pixels_to_corrupt = int(nivel_ruido * height * width)
    indices_to_corrupt = np.random.choice(height * width, num_pixels_to_corrupt, replace=False)
    noisy_pixels = np.random.randint(0, 256, num_pixels_to_corrupt)  # Generar valores de ruido para los píxeles corruptos
    noisy_image[np.unravel_index(indices_to_corrupt, (height, width))] = noisy_pixels  # Aplicar ruido a los píxeles seleccionados
    return noisy_image

def ruido_impulsivo_unimodal(shape, prob=0.05, valor=1):
    filas, columnas = shape
    
    imagen_ruidosa = np.zeros(shape, dtype=np.uint8)
    
    mascara_ruido = np.random.random((filas,columnas)) < prob

    imagen_ruidosa[mascara_ruido] = valor

    return imagen_ruidosa

def ruido_gaussiano(shape, media=0, desviacion_estandar=10):
    fila, columna = shape
    
    ruido_gaussiano = np.random.normal(media, desviacion_estandar, (fila, columna))
    
    ruido_gaussiano = ruido_gaussiano - np.mean(ruido_gaussiano)
    
    return ruido_gaussiano

def prewitt_deteccion_bordes(imagen, umbral=0.1):  
    kernel_x = np.array([[-1, 0, 1],
                         [-1, 0, 1],
                         [-1, 0, 1]])

    kernel_y = np.array([[-1, -1, -1],
                         [0, 0, 0],
                         [1, 1, 1]])

    gradiente_x = convolve2d(imagen, kernel_x, mode='same', boundary='symm')
    gradiente_y = convolve2d(imagen, kernel_y, mode='same', boundary='symm')
    
    magnitud_gradiente = np.abs(gradiente_x) + np.abs(gradiente_y)
    
    edge_imagen = np.zeros_like(magnitud_gradiente)
    edge_imagen[magnitud_gradiente > umbral] = 1
    
    return edge_imagen

def laplaciano_deteccion_bordes(imagen, ddepth, ksize):
    laplacian_image = cv.Laplacian(imagen, ddepth, ksize=ksize)
    return laplacian_image
  
def trackbar_transformacion(imagen, variables_trackbar, parametros_trackbar, transformacion):
    imagen_original = imagen.copy()
    NOMBRE_VENTANA = 'Trackbars'
    def on_trackbar(value):
        valores_trackbar = []
        for i in range(len(variables_trackbar)):
            valores_trackbar.append(cv.getTrackbarPos(variables_trackbar[i], NOMBRE_VENTANA))
        # valores_trackbar = [cv.getTrackbarPos(var, NOMBRE_VENTANA) for var in variables_trackbar]
        
        imagen_transformada = transformacion(imagen_original, valores_trackbar)

        imagen_transformada_normalizada = cv.normalize(imagen_transformada,None,0,255,cv.NORM_MINMAX)

        imagen_resultado = cv.convertScaleAbs(imagen_transformada_normalizada)

        cv.imshow('Imagen Original', imagen)
        cv.imshow('Imagen Transformada', imagen_resultado)

    cv.namedWindow(NOMBRE_VENTANA)

    for i, var in enumerate(variables_trackbar):
        cv.createTrackbar(var, NOMBRE_VENTANA, parametros_trackbar[i][0], parametros_trackbar[i][1], on_trackbar)

    on_trackbar(0)

    cv.waitKey(0)
    cv.destroyAllWindows()

mosquito = cv.imread("Imagenes/mosquito.jpg", cv.IMREAD_GRAYSCALE)

In [ ]:

variables_trackbar = ['desviacionSTD', 'valor','tipo dato','ksize', 
                        'umbral_min', 'umbral_max', 'L2Grad','dx', 'dy','probabilidad_impulsivo','metodo']
parametros_trackbar = [[0,255],[0,255],[1,2], [1,31], [0,255], [0,255], [0,1],[0,1],[0,1],[1,100],[0,3]]
shape_mosquito = mosquito.shape
def transformacion(imagen, valores_trackbar):
    global shape_mosquito
    stdv = valores_trackbar[0]
    valor = valores_trackbar[1]
    tipo_dato = valores_trackbar[2]
    ksize = valores_trackbar[3]-1
    if tipo_dato == 1:
        ddepth = cv.CV_8U
    else:
        ddepth = cv.CV_64F
    if ksize % 2 == 0:
        ksize += 1
    umbral_minimo = valores_trackbar[4]
    umbral_maximo = valores_trackbar[5]
    L2gradient = valores_trackbar[6]
    dx = np.max(valores_trackbar[7],0)
    dy = np.max(valores_trackbar[8],0)
    prob = valores_trackbar[9]/100
    metodo = valores_trackbar[10]
    if L2gradient == 1:
        gradiente = True
    else:
        gradiente = False
    if dx + dy == 0:
        dx += 1
    ruido_g = ruido_gaussiano(shape_mosquito, 0, stdv).astype(np.uint8)
    ruido_u = ruido_impulsivo_unimodal(shape_mosquito, prob, 1).astype(np.uint8)
    ruidos = cv.add(ruido_g, ruido_u)
    imagen_ruido = cv.add(imagen, ruidos).astype(np.uint8)

    if metodo == 0:
        salida = sobel_deteccion_bordes(imagen_ruido, ddepth, dx, dy, ksize)
    elif metodo == 1:
        salida = cv.Canny(imagen_ruido,umbral_minimo, umbral_maximo, gradiente)
    elif metodo == 2:
        salida = prewitt_deteccion_bordes(imagen_ruido, 0.1)
    else:
        salida = laplaciano_deteccion_bordes(imagen_ruido, ddepth, ksize)
    return salida

trackbar_transformacion(mosquito, variables_trackbar, parametros_trackbar, transformacion)

*Ignorar posibles errores obtenidos durante la ejecución en las trackbars 